- 별도의 API를 제공해주지 않는 경우 스크래핑을 활용하여 홈페이지를 통해 긁어와야 할 수 있음.

- 우리가 활용할 웹페이지: KRIC 철도통계, 철도운영현황>도시철도여객수송>역별 승강차실적>서울메트로,2020년의 표
- url: http://www.kric.go.kr/jsp/industry/rss/citystapassList.jsp?q_org_cd=A010010021&q_fdate=2020
- 활용하는 경우: if 상사가 "도시철도여객수송의 역별 승강차실적 2020 서울메트로 표 보고 '역별 승하차인원'을 엑셀로 정리해주세요"라고 요구한다면 페이지보고 왔다갔다 하면서 정리하지말고 이렇게 하기

#1. Scraping
- 내가 원하는 정보가 있는 url에서 html코드(페이지 소스)를 긁어 오는 '데이터 가져오기' 작업
- tag soup를 가져오는 작업
- 활용 라이브러리: requests

In [31]:
import requests     # request를 활용하여 url 가져오기 
url = "http://www.kric.go.kr/jsp/industry/rss/citystapassList.jsp?q_org_cd=A010010021&q_fdate=2020"       # url 주소 저장

In [32]:
html_text = requests.get(url).text      # 위 url에서 html코드(페이지소스)를 모두 긁어오는 명령. GET 방식 활용

In [33]:
print(html_text)















<!DOCTYPE HTML>















 
 

<html lang="ko">
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="autocomplete" content="off" />
<title>철도통계 &gt; 철도운영현황 &gt; 도시철도여객수송 &gt; 역별 승강차실적(월) - 111철도산업정보센터</title>
<link rel="stylesheet" type="text/css" href="/css/font.css">
<link rel="shortcut icon" href="/images/kric.ico" />
<link rel="apple-touch-icon" href="/images/favicon.png" />
<script type="text/javascript" src="/ext/script/JControl.js"></script>
<script type="text/javascript" src="/ext/script/JDom.js"></script>
<script type="text/javascript" src="/ext/script/JForm.js"></script>
<script type="text/javascript" src="/ext/script/JPopup.js"></script>
<script type="text/javascript" src="/ext/script/JPrototype.js"></script>
<script type="text/javascript" src="/ext/script/JUtil.js"></script>
<script type="text/javascript" src="/ext/script/JValidation.js"></script>
<script type="text/javascript" src="/ext/script/JUpload.js"></sc

#2. Parsing
- 내가 원하는 정보만 뽑아내는 '데이터 구분' 작업
- 하나하나 단위릘 쪼개서 원하는 데이터를 찾아갈 수 있도록 해주는 작업
- 활용 라이브러리: BeautifulSoup

In [34]:
from bs4 import BeautifulSoup       # BeautifulSoup 라이브러리 불러오기 

In [35]:
# 어떤 것을 parser하겠는가?의 항목
soup = BeautifulSoup(html_text, "html.parser")      # 가져온 html_text를 html이라는 형식으로 parser 하라는 명령

In [36]:
print(soup)     # Scraping에서 print(html_text)와 비교해보기 
                # 강사님 말씀으로는 위와는 다르게 계층별로 나누어져 parsing이 된 상태라고 함 


<!DOCTYPE HTML>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible">
<meta content="off" name="autocomplete"/>
<title>철도통계 &gt; 철도운영현황 &gt; 도시철도여객수송 &gt; 역별 승강차실적(월) - 111철도산업정보센터</title>
<link href="/css/font.css" rel="stylesheet" type="text/css"/>
<link href="/images/kric.ico" rel="shortcut icon">
<link href="/images/favicon.png" rel="apple-touch-icon"/>
<script src="/ext/script/JControl.js" type="text/javascript"></script>
<script src="/ext/script/JDom.js" type="text/javascript"></script>
<script src="/ext/script/JForm.js" type="text/javascript"></script>
<script src="/ext/script/JPopup.js" type="text/javascript"></script>
<script src="/ext/script/JPrototype.js" type="text/javascript"></script>
<script src="/ext/script/JUtil.js" type="text/javascript"></script>
<script src="/ext/script/JValidation.js" type="text/javascript"></script>
<script src="/ext/script/JUpload.js" type="text/javascript"></script>
<script src="/resource/script/

#3. Manipulation
- 이제 BeautifulSoup으로 잘 나눠진 html 코드를 이용하여 원하는 데이터만을 가져오는 "데이터 가공" 실시
- 우리는 해당 페이지에 보이는 표의 '서울역~12월'에 해당하는 내용을 각 역마다 데이터를 가져올 것임


##1) tab: 표가 포함된 "listtbl_c100"에 대한 내용 가져오기

[변수 정리]
- tab: 표가 포함된 "listtbl_c100"만을 가져온 변순
- trs: 표에서 thead가 아닌 tbody만을 가져온 변수 
- tdcols: tdcol을 find로 찾아 할당한 변수(1개만 할당)
- tds: 해당 표의 'td'에 해당하는 모든 값들을 가져온 변수
- stationpassengers: 각 별 별로 '역(station)-승차인원 합계(ride)-하차인원 합계(alight)) 를 dictionary 형태로 저장한 변수

In [37]:
'''
- url에 들어가보면 바로 있는 '역별 승강차실적(월)'에 해당하는 table을 말한다. 
- 'Ctrl+F'>listtbl_c100 검색해보기
'''
tab = soup.find("table", {"class":"listtbl_c100"})      # table 찾아와 tab 변수에 넣기
                                                        # 이때, 이름이 "listtbl_c100"인 table을 가져오는 것

In [38]:
print(tab)      # 위에서 출력된 soup와는 다르게 원하는 table에 해당하는 코드만 떼어 냄.
                

<table class="listtbl_c100" style="width:2600px" summary="조회연도별 기관별 도시철도 역별 승강차실적(월)">
<caption class="hidden">역별 승강차실적(월)</caption>
<thead>
<tr>
<th class="btmbd_c" rowspan="2">역명 </th>
<th class="btmbd_c" rowspan="2">호(선)명  </th>
<th class="btmbd_g" colspan="2">합계</th>
<th class="btmbd_g" colspan="2">1월</th>
<th class="btmbd_g" colspan="2">2월</th>
<th class="btmbd_g" colspan="2">3월</th>
<th class="btmbd_g" colspan="2">4월</th>
<th class="btmbd_g" colspan="2">5월</th>
<th class="btmbd_g" colspan="2">6월</th>
<th class="btmbd_g" colspan="2">7월</th>
<th class="btmbd_g" colspan="2">8월</th>
<th class="btmbd_g" colspan="2">9월</th>
<th class="btmbd_g" colspan="2">10월</th>
<th class="btmbd_g" colspan="2">11월</th>
<th class="btmbd_g" colspan="2">12월</th>
</tr>
<tr>
<th class="btmbd_c">승차인원(명) </th>
<th class="btmbd_c">하차인원(명) </th>
<th class="btmbd_c">승차인원(명) </th>
<th class="btmbd_c">하차인원(명) </th>
<th class="btmbd_c">승차인원(명) </th>
<th class="btmbd_c">하차인원(명) </th>
<th class="btmbd_c">승차인원(명) </

##2) tbody만 잘라내기
표에서 머리에 해당하는 내용말고 body(tbody)에 해당하는 '역-월별 합계' 데이터만 list형태로 가져오기

In [56]:
trs = tab.find('tbody').find_all('tr')      # tbody에 해당하는 내용만 trs에 저장하기
trs                                         # 1page에 나오는 "합계" 행부터 '서울역~을지로4가'에 해당하는 내용(tbody 내용)이 trs에 할당 된 것을 알 수 있음.

[<tr>
 <td class="tdcol" colspan="2">합 계</td>
 <td class="tdrcol">750,053,400</td>
 <td class="tdrcol">755,667,165</td>
 <td class="tdrcol">82,940,936</td>
 <td class="tdrcol">83,639,554</td>
 <td class="tdrcol">67,303,167</td>
 <td class="tdrcol">67,784,926</td>
 <td class="tdrcol">54,355,497</td>
 <td class="tdrcol">54,723,582</td>
 <td class="tdrcol">57,778,435</td>
 <td class="tdrcol">58,216,579</td>
 <td class="tdrcol">63,613,539</td>
 <td class="tdrcol">64,074,781</td>
 <td class="tdrcol">66,375,425</td>
 <td class="tdrcol">66,911,180</td>
 <td class="tdrcol">61,780,698</td>
 <td class="tdrcol">62,438,093</td>
 <td class="tdrcol">58,994,026</td>
 <td class="tdrcol">59,449,165</td>
 <td class="tdrcol">54,776,500</td>
 <td class="tdrcol">55,114,176</td>
 <td class="tdrcol">64,014,692</td>
 <td class="tdrcol">64,650,093</td>
 <td class="tdrcol">65,523,503</td>
 <td class="tdrcol">66,008,324</td>
 <td class="tdrcol">52,596,982</td>
 <td class="tdrcol">52,656,712</td>
 </tr>, <tr>
 <t

In [41]:
# 데이터 1행 보기: '서울역' 데이터
trs[1]

<tr>
<td class="tdcol">서울역</td>
<td class="tdcol">1호선</td>
<td class="tdr">12,785,590</td>
<td class="tdr">12,150,332</td>
<td class="tdr">1,656,228</td>
<td class="tdr">1,541,934</td>
<td class="tdr">1,165,552</td>
<td class="tdr">1,105,578</td>
<td class="tdr">820,290</td>
<td class="tdr">780,839</td>
<td class="tdr">919,812</td>
<td class="tdr">883,313</td>
<td class="tdr">1,086,363</td>
<td class="tdr">1,026,246</td>
<td class="tdr">1,127,907</td>
<td class="tdr">1,073,229</td>
<td class="tdr">1,049,008</td>
<td class="tdr">1,015,433</td>
<td class="tdr">1,015,722</td>
<td class="tdr">970,546</td>
<td class="tdr">865,744</td>
<td class="tdr">844,702</td>
<td class="tdr">1,093,706</td>
<td class="tdr">1,027,577</td>
<td class="tdr">1,139,033</td>
<td class="tdr">1,079,034</td>
<td class="tdr">846,225</td>
<td class="tdr">801,901</td>
</tr>

## 3) table의 column에 해당하는 값만(tdcols[서울역, 1호선 등]) 잘라내기

In [ ]:
tdcols = trs[1].find("td", {"class":"tdcol"})       # tdcols: trs에서 "tdcol"에 해당하는 "서울역"을 할당한 변수
tdcols                                              # find 명령은 가장 먼저 찾는 1개만 찾기 때문에 "1호선"은 할당되지 않은 것을 알 수 있다. 

## 4) 여러개의 "td" 값들을 전부 가져오기

In [57]:
tds = trs[1].find_all('td')     # tds: 해당 표의 'td'에 해당하는 모든 값들을 가져오기 
tds                             # .find_all() 메서드 활용

[<td class="tdcol">서울역</td>,
 <td class="tdcol">1호선</td>,
 <td class="tdr">12,785,590</td>,
 <td class="tdr">12,150,332</td>,
 <td class="tdr">1,656,228</td>,
 <td class="tdr">1,541,934</td>,
 <td class="tdr">1,165,552</td>,
 <td class="tdr">1,105,578</td>,
 <td class="tdr">820,290</td>,
 <td class="tdr">780,839</td>,
 <td class="tdr">919,812</td>,
 <td class="tdr">883,313</td>,
 <td class="tdr">1,086,363</td>,
 <td class="tdr">1,026,246</td>,
 <td class="tdr">1,127,907</td>,
 <td class="tdr">1,073,229</td>,
 <td class="tdr">1,049,008</td>,
 <td class="tdr">1,015,433</td>,
 <td class="tdr">1,015,722</td>,
 <td class="tdr">970,546</td>,
 <td class="tdr">865,744</td>,
 <td class="tdr">844,702</td>,
 <td class="tdr">1,093,706</td>,
 <td class="tdr">1,027,577</td>,
 <td class="tdr">1,139,033</td>,
 <td class="tdr">1,079,034</td>,
 <td class="tdr">846,225</td>,
 <td class="tdr">801,901</td>]

In [58]:
tds[0]

<td class="tdcol">서울역</td>

##5) tds[0]에서 "서울역"에 해당하는 부분만을 가져오기

In [61]:
tds[0].text     # 메서드 .text 활용

'서울역'

In [62]:
tds[1].text     # 같은 행에 적용도 가져오기

'1호선'

In [52]:
tds[2].text     # '합계'의 "승차인원"

'12,785,590'

In [53]:
tds[3].text     # '합계'의 "하차인원"

'12,150,332'

## 6) 여러개의 역에 대해 Manipulation을 해보자

In [63]:
stationpassengers = []
for tr in trs[1:]:
    dic = {}
    tds = tr.find_all('td')
    dic['station'] = tds[0].text    # 'station': 역명 (홈페이지 표 참고)
    dic['ride'] = tds[2].text       # 'ride': 승차인원 
    dic['alight'] = tds[3].text     # 'alight': 하차인원 
    stationpassengers.append(dic)

In [64]:
stationpassengers

[{'alight': '12,150,332', 'ride': '12,785,590', 'station': '서울역'},
 {'alight': '6,069,179', 'ride': '6,021,911', 'station': '시청'},
 {'alight': '10,220,308', 'ride': '10,487,811', 'station': '종각'},
 {'alight': '7,490,908', 'ride': '7,973,503', 'station': '종로3가'},
 {'alight': '6,659,877', 'ride': '6,634,179', 'station': '종로5가'},
 {'alight': '3,739,510', 'ride': '3,657,095', 'station': '동대문'},
 {'alight': '4,187,510', 'ride': '4,247,367', 'station': '신설동'},
 {'alight': '5,430,094', 'ride': '5,291,736', 'station': '제기동'},
 {'alight': '6,495,649', 'ride': '6,483,701', 'station': '청량리(지하)'},
 {'alight': '3,037,693', 'ride': '2,911,852', 'station': '동묘앞'},
 {'alight': '6,322,543', 'ride': '6,462,974', 'station': '시청'},
 {'alight': '11,557,833', 'ride': '11,361,519', 'station': '을지로입구'},
 {'alight': '5,894,217', 'ride': '5,901,229', 'station': '을지로3가'},
 {'alight': '3,632,367', 'ride': '3,661,188', 'station': '을지로4가'}]